# Factored Workshops! 🚀
_Crearás un modelo capaz de predecir la duración total de un viaje de taxi en Nueva York 🗽_

# Pipelines en scikit-learn

El objetivo de esta sección es mostrar cómo se pueden desarrollar flujos de preprocesamiento en sklearn. Vamos a explorar las diferentes operaciones que se le pueden hacer a los diferentes tipos de variables y cómo agrupar todas las operaciones en un solo elemento de sklearn que tenga los métodos .fit() y .transform().

In [44]:
import numpy as np
import pandas as pd

from typing import List
from sklearn import set_config
set_config(display='diagram')

In [2]:
data = pd.read_csv(
    "https://factored-workshops.s3.amazonaws.com/taxi-trip-duration.csv"
)

data.shape

(1458644, 13)

In [3]:
# Limitar rango de datos
tiempo_minimo = 60 # 1 minuto
tiempo_maximo = 36000 # 10 horas

data = data[
    (data["trip_duration"] > tiempo_minimo) &
    (data["trip_duration"] < tiempo_maximo)
]
data.shape

(1447855, 13)

In [5]:
data.head(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_borough,dropoff_borough
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,Manhattan,Manhattan
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,Manhattan,Brooklyn
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,Manhattan,Brooklyn


Es importante siempre separar la variable dependiente—en este caso **trip_duration** del dataframe que vamos a usar para crear las variables independientes.

In [8]:
y = data["trip_duration"]
x = data.drop(
    ["id", "trip_duration", "dropoff_datetime", "store_and_fwd_flag"],
    axis="columns"
)

# Quitamos:
# id -> Es un identificador unico y este no brinda nada de informacion
# trip_duration -> Variable a predecir
# dropoff_datetime -> Se elimina pq para nuevos datos no se tendra este dato
# store_and_fwd_flag -> Variable bandera de si almacenar o no el viaje

## División de los datos

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    random_state=0)
# test_size=0.25 -- default

In [12]:
# help(train_test_split)

## Transformers

`scikit-learn` incluye una gran [lista](https://scikit-learn.org/stable/data_transforms.html) de transformers que nos **permiten limpiar y transformar datos** dependiendo de nuestro objetivo. Sin importar el tipo de transformacion que apliquen, todos los transformers siguen la convención de tener por lo menos dos métodos:

- `.fit()` calcula los parámetros necesarios para realizar la transformación a partir de unos datos de entrada. Este método se ejecuta **únicamente en los datos de entrenamiento** para asegurarnos que los parámetros no contienen información de los datos de validación.
- `.transform()` aplica la transformación a los datos.

Veamos un ejemplo usando `StandardScaler`, un transformer que nos permite **remover la media y escalar** los datos para que **tengan varianza de 1**. Vamos a usarlo para normalizar los coordenadas de inicio del viaje.

In [15]:
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler()
transformer.fit(
    x_train[["pickup_longitude", "pickup_latitude"]]
)
normed_array = transformer.transform(
    x_test[["pickup_longitude", "pickup_latitude"]]
)
print(normed_array)

[[-0.30923835 -0.27478234]
 [-0.13589444  0.75675283]
 [ 0.24599071  0.58169127]
 ...
 [-0.19727957 -0.13002453]
 [-0.03955223 -0.08161096]
 [-0.11150857 -0.79791458]]


## Custom Transformers

A pesar de que scikit-learn ofrece varias operaciones para transformar datos, frecuentemente necesitamos crear transformaciones que son específicas a nuestro proyecto. Para esto vamos a aprender cómo implementar custom transformers.

Todos los transformer custom deben heredar `BaseEstimator` y `TransformerMixin` para que tengan todas las funciones necesarias para conectarse a otros objetos de sklearn. Por convención de sklearn, los objetos usados para transformar datos siempre deben tener los métodos `.fit()` y `.transform()`. Ambos métodos reciben X y y para que se integren sin problemas a pipelines de sklearn.

- El método `.fit()` sirve para almacenar cantidades que vamos a usar durante la transformación de los datos y siempre debe retornar self. 
- El método `.transform()` ejecuta la transformación y retorna los datos transformados.

Ahora vamos a replicar el StandardScaler pero esta vez escribiéndolo como un transformer personalizado que no retorne un array sino un DataFrame.

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin

class PrimerTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mean = X.mean()
        self.std = X.std()
        return self

    def transform(self, X, y=None):
        return (X - self.mean) / self.std

## Checkpoint #1

In [19]:
pt = PrimerTransformer()

pt.fit(x_train[["pickup_longitude", "pickup_latitude"]])
pt.transform(x_test[["pickup_longitude", "pickup_latitude"]])

,pickup_longitude,pickup_latitude
5949,-0.309238,-0.274782
255492,-0.135894,0.756752
979171,0.245991,0.581691
121356,0.176797,0.385864
524382,0.194096,0.182913
...,...,...
885062,-0.488949,-0.292409
755237,-0.565470,-1.408094
790367,-0.197279,-0.130024
106875,-0.039552,-0.081611


En este [link](https://scikit-learn.org/stable/developers/develop.html) pueden encontrar más detalles de qué papel juegan los métodos __init__, .fit() y .transform()

## Transformer Fechas

Ahora que sabemos cómo construir objetos para transformar datos, vamos a **crear un transformer para crear variables como el día de la semana y la hora** del momento en el que empieza el servicio.
Como vimos la semana pasada en nuestro EDA, esa información puede ser relevante para determinar la duración del viaje.

En nuestra transformación no debemos almacenar ningún dato para hacer las transformación entonces dejamos el método .fit() vacío. En .transform() extraemos los datos de fecha que nos interesan y retornamos un dataframe con las nuevas variables.

In [28]:
class TransformerFechas(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        columna_fecha = pd.to_datetime(X["pickup_datetime"])
        fecha_df = pd.DataFrame()
        # TODO: Crear columnas con dia de la semana y hora de recogida.
        fecha_df['weekday'] = columna_fecha.dt.weekday
        fecha_df['hour'] = columna_fecha.dt.hour
        return fecha_df

## Checkpoint #2.1

In [29]:
transformer_fechas = TransformerFechas()
fechas_df = transformer_fechas.fit_transform(x_train)
fechas_df.head()

,weekday,hour
518949,3,21
1128931,6,21
574396,1,18
54790,6,17
599130,0,16


In [31]:
x_train["pickup_datetime"].head(3)

518949     2016-06-02 21:41:39
1128931    2016-03-27 21:59:43
574396     2016-02-02 18:13:10
Name: pickup_datetime, dtype: object

## Transformer Distancia

También queremos crear una feature que nos ayude a medir la distancia entre el punto de origen y el punto de destino usando la longitud y la latitud en los datos. 
Nuevamente no tenemos que almacenar cantidades en nuestro método .fit() y calculamos la distancia entre los dos puntos usando la [distancia de Haversine](https://es.wikipedia.org/wiki/F%C3%B3rmula_del_semiverseno). 

En el código ya está implementada la función para calcular la distancia y no es necesario fijarse en los detalles de la implementación. Como pueden ver, nuestros transformers pueden incluir funciones adicionales que nos ayuden a calcular cantidades útiles para realizar la transformación.

In [33]:
class TransformerDistancia(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_init = X[["pickup_latitude", "pickup_longitude"]].to_numpy()
        X_final = X[["dropoff_latitude", "dropoff_longitude"]].to_numpy()

        # Distancia de Haversine
        # TODO: Calcular la variable distancia usando la funcion
        # distancia de Haversine.
        distancia = self.distancia_haversine(X_init,X_final)
        distancia_df = pd.DataFrame()
        distancia_df["distancia"] = distancia
        return distancia_df
    
    # DISTANCIA GEOESPACIALES - LATITUD/LONGITUD
    def distancia_haversine(self, X_init, X_final):
        # Convertir de decimal a radianes
        X_init = np.radians(X_init)
        X_final = np.radians(X_final)

        # Formula Haversine
        dlat = X_final[:, 0] - X_init[:, 0] 
        dlon = X_final[:, 1] - X_init[:, 1]
        a = np.sin(dlat / 2) ** 2 + np.cos(X_init[:, 0]) * (
            np.cos(X_final[:, 0])) * (
            np.sin(dlon / 2) ** 2)
        c = 2 * np.arcsin(np.sqrt(a))
        r = 6371 
        # Radius of earth in kilometers. Use 3956 for miles. 
        # Determines return value units.
        return c * r

## Checkpoint #2.2

In [34]:
transformer_dist = TransformerDistancia()
distancias_df = transformer_dist.fit_transform(train_df)
distancias_df.head()

,distancia
0,2.404355
1,0.390267
2,5.629826
3,4.298386
4,7.488963


## Unión de transformers con Pipelines

## **Pipeline Numérico**

Ahora vamos a usar los objetos de `Pipeline` y `ColumnTransformer` para unir los transformers que ya hemos creado con otros disponibles en sklearn y lograr todas las transformaciones que queremos realizar a todos los datos.

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

`ColumnTransformer` nos permite **elegir** las **** sobre las que queremos **aplicar una transformación** cuando nos llegan columnas adicionales. En este caso queremos que al TransformerDistancia llegue únicamente pickup_longitude, pickup_latitude, dropoff_longitude y dropoff_latitude .

Leyendo la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) sabemos que **debemos pasar una tupla** con el _nombre del transformer_, la _clase_ que define el transformer y las _columnas_ sobre las que queremos _aplicar la transformación_. 

`ColumnTransformer` también nos permite definir qué se debe hacer con las **columnas** que **no** estamos **transformando**; en este caso elegimos **pasarlas sin transformarlas** `remainder="passthrough"`.

In [46]:
coord_cols = [
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude"
]

transformer_coord = ColumnTransformer(
    [
        ("transformer_dist", TransformerDistancia(), coord_cols),
    ],
    remainder="passthrough" # Las demas variables no sean transformadas
)

display(transformer_coord)

ColumnTransformer(remainder='passthrough',
                  transformers=[('transformer_dist', TransformerDistancia(),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude'])])

Ahora que tenemos nuestro transformer que realiza transformación usando las coordenadas y deja pasar columnas adicionales, vamos a usarlo para unirlo con la otra variable númerica que tenemos en el dataset `—passenger_count—` y normalizarlas usando `StandardScaler`. Para esto vamos a usar el objeto `Pipeline`.

`Pipeline` [(documentación Pipeline)](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) nos permite concatenar transformaciones de sklearn. En este caso vamos a concatenar el ColumnTransformer que creamos con TransformerDistancia con el StandardScaler. Como passenger_count no estaba entre columnas que selecciona transformer_coord, esa columna pasa directamente a ser normalizada por el StandardScaler.

In [41]:
num_cols = ["passenger_count"] + coord_cols

num_pipeline = Pipeline(
    [
        ("transformer_coord", transformer_coord), # distancia / los 2 puntos
        ("scaler", StandardScaler())
    ]
)

X_num = num_pipeline.fit_transform(x_train[num_cols], y_train)
print(X_num)

[[-0.25154928 -0.50616698]
 [-0.73141214  0.25385023]
 [ 0.51692963 -0.50616698]
 ...
 [-0.41058443 -0.50616698]
 [-0.3461049  -0.50616698]
 [ 1.33898968  0.25385023]]


Usando la función `display` podemos ver el diagrama del flujo que acabamos de crear. Por `TransformerDistancia` pasan las variables de longitud y latitud y por `passthrough` pasa cualquier columna que le pasemos al ColumnTransformer que no sean longitud o latitud—en este caso passenger_count. El último paso es aplicar un `StandardScaler` a la variable de distancia que sale de TransformerDistancia y a las variables que vienen de passthrough.

In [45]:
display(num_pipeline)

Pipeline(steps=[('transformer_coord',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transformer_dist',
                                                  TransformerDistancia(),
                                                  ['pickup_longitude',
                                                   'pickup_latitude',
                                                   'dropoff_longitude',
                                                   'dropoff_latitude'])])),
                ('scaler', StandardScaler())])

## **Pipeline Categórico**
Usaremos una lógica similar para las variables categóricas pero esta vez para concatenar el resultado de extraer variables temporales de la fecha de inicio del viaje con un `OneHotEncoder` para las variables categóricas de los datos.

In [53]:
from sklearn.preprocessing import OrdinalEncoder

cat_cols = ["vendor_id", "pickup_borough", "pickup_datetime"]

transformer_fechas = ColumnTransformer(
    [
        #TODO: Punto 1 de Checkpoint 3
        ("transformer_fechas",TransformerFechas(),['pickup_datetime'])
    ],
    remainder="passthrough"
)

cat_pipeline = Pipeline(
    [
        #TODO: Punto 2 de Checkpoint 3
        ("transformer_fechas",transformer_fechas),
        ("OrdinalEncoder",OrdinalEncoder())
    ]
)

X_cat = cat_pipeline.fit_transform(x_train[cat_cols])

In [54]:
display(cat_pipeline)

Pipeline(steps=[('transformer_fechas',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transformer_fechas',
                                                  TransformerFechas(),
                                                  ['pickup_datetime'])])),
                ('OrdinalEncoder', OrdinalEncoder())])

## Unión de Pipelines

Por último, usaremos nuevamente el `ColumnTransformer` para determinar cuáles son las variables numéricas y las variables catégoricas en nuestros datos. 
Definiendo varios elementos en la lista que le pasamos al ColumnTransformer le hacemos saber a sklearn que queremos diferentes transformaciones para las columnas y al final queremos unirlas para que todas queden en un solo numpy array. En nuestro caso, unimos las columnas que resultan del preprocesamiento de las categóricas y del de las numéricas.

In [55]:
from sklearn.pipeline import FeatureUnion

full_pipeline = ColumnTransformer(
    [
        ("num_pipeline", num_pipeline, num_cols),
        ("cat_pipeline", cat_pipeline, cat_cols)
    ]
)

X_transformed = full_pipeline.fit_transform(x_train, y_train)
print(X_transformed.shape)

(1085891, 6)


En este diagrama podemos ver que ejecutamos las transformaciones por separado para las variables numéricas y las categóricas y al final las unimos para tener los datos en un array que le pasaremos al modelo al momento de entrenar.

In [56]:
display(full_pipeline)

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('transformer_coord',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('transformer_dist',
                                                                                   TransformerDistancia(),
                                                                                   ['pickup_longitude',
                                                                                    'pickup_latitude',
                                                                                    'dropoff_longitude',
                                                                                    'dropoff_latitude'])])),
                                                 ('scaler', StandardScaler())]),
                                 ['passenger_count', 'pickup_longitude',
                                  'pickup_latitude', 'dropoff_longitude',
                                  'dropoff_latitude']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('transformer_fechas',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('transformer_fechas',
                                                                                   TransformerFechas(),
                                                                                   ['pickup_datetime'])])),
                                                 ('OrdinalEncoder',
                                                  OrdinalEncoder())]),
                                 ['vendor_id', 'pickup_borough',
                                  'pickup_datetime'])])